In [1]:
import pandas as pd
import glob

In [2]:
paths = glob.glob('../data/namwon/*.csv')

In [6]:
path1 = '../data/namwon/전라북도 남원시_문화및집회시설_20230822.csv'
path2 = '../data/namwon/전라북도 남원시_사회복지시설현황_20231106.csv'
path3 = '../data/namwon/전라북도 남원시_작은 도서관 현황_20231106.csv'
path4 =  '../data/namwon/전북특별자치도 남원시_숙박업소_20240508.csv'

In [7]:
def split_address(address):
    parts = address.split()
    
    if len(parts) >= 4:
        city_name = parts[0]  # 예: 전라북도
        district = parts[1]  # 예: 남원시
        town = parts[2]  # 예: 대강면
        road_name = ' '.join(parts[3:-1])  # 예: 대강월산길 (주소의 마지막 요소 전까지 모두 도로명으로 처리)
        bungee = parts[-1]  # 예: 37-16 (주소의 마지막 요소는 항상 번지로 처리)
        
        return city_name, district, town, road_name, bungee
    else:
        return None, None, None, None, None

In [8]:
for idx, val in enumerate(paths):
    print(f"{idx}: {val}")

0: ../data/namwon\welfare_facilities.csv
1: ../data/namwon\전라북도 남원시_문화및집회시설_20230822.csv
2: ../data/namwon\전라북도 남원시_빈집_20230824.csv
3: ../data/namwon\전라북도 남원시_사회복지시설현황_20231106.csv
4: ../data/namwon\전라북도 남원시_인구현황_20230801.csv
5: ../data/namwon\전라북도 남원시_작은 도서관 현황_20231106.csv
6: ../data/namwon\전북특별자치도 남원시_숙박업소_20240508.csv


In [10]:
# 0, 2, 4, 5
culture_facilities = pd.read_csv(path1, encoding='cp949') # 문화 및 집회 시설
social_welfare = pd.read_csv(path2, encoding='cp949') # 사회복지시설
library = pd.read_csv(path3, encoding='cp949') # 도서관
accommodation = pd.read_csv(path4, encoding='cp949') # 숙박업소

In [11]:
culture_facilities[['시도', '시군구', '읍면동', '도로명', '번지']] = culture_facilities['소재지'].apply(lambda x: pd.Series(split_address(x)))
social_welfare[['시도', '시군구', '읍면동', '도로명', '번지']] = social_welfare['도로명주소'].apply(lambda x: pd.Series(split_address(x)))
library[['시도', '시군구', '읍면동', '도로명', '번지']] = library['위치'].apply(lambda x: pd.Series(split_address(x)))
accommodation[['시도', '시군구', '읍면동', '도로명', '번지']] = accommodation['소재지'].apply(lambda x: pd.Series(split_address(x)))

In [12]:
filtered_culture_facilities = culture_facilities.loc[:, ['소재지', '명칭', '읍면동']].rename(columns={'명칭': '시설명'})
filtered_culture_facilities['구분'] = '문화시설'
filtered_culture_facilities['num'] = 1

filtered_accommodation = accommodation.loc[:, ['소재지', '숙박업소명', '읍면동']].rename(columns={'숙박업소명': '시설명'})
filtered_accommodation['구분'] = '숙박업소'
filtered_accommodation['num'] = 1

filtered_library = library.loc[:, ['위치', '도서관명', '읍면동']].rename(columns={'위치': '소재지', '도서관명': '시설명'})
filtered_library['구분'] = '도서관'
filtered_library['num'] = 1

filtered_social_welfare = social_welfare.loc[:, ['도로명주소', '시설명', '읍면동']].rename(columns={'도로명주소': '소재지'})
filtered_social_welfare['구분'] = '사회복지시설'
filtered_social_welfare['num'] = 1

In [13]:
df = pd.concat([filtered_culture_facilities, filtered_accommodation, filtered_library, filtered_social_welfare])
df[:3]

,소재지,시설명,읍면동,구분,num
0,전라북도 남원시 양림길 43,춘향문화예술회관,양림길,문화시설,1
1,전라북도 남원시 광한북로 54,지리산소극장,광한북로,문화시설,1
2,전라북도 남원시 양림길 54,국립민속국악원,양림길,문화시설,1


In [14]:
df.to_csv('../data/namwon/welfare_facilities.csv', index=False)